Most of below migrated into cloud cover function in functions.py (same directory)

In [2]:
# Now to iterate for a few years and see what happens
import os
os.chdir('/home/KID/KID_scripts/cloud_mapping/')
from functions import cloudcover_func

In [3]:
for i in range(0,5):
    try:
        cloudcover_func(i*5, 2018, footprint_path='map.geojson')
    except:
        pass

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Querying products: 100%|██████████| 1897/1897 [00:19<00:00, 91.94product/s] 


In [4]:
!conda info
# To kill port in use (only do if you know its n old juyter instance)
# fuser 8888/tcp


     active environment : None
       user config file : /home/jovyan/.condarc
 populated config files : /opt/conda/.condarc
                          /home/jovyan/.condarc
          conda version : 22.11.1
    conda-build version : not installed
         python version : 3.10.8.final.0
       virtual packages : __archspec=1=x86_64
                          __glibc=2.35=0
                          __linux=5.15.0=0
                          __unix=0=0
       base environment : /opt/conda  (writable)
      conda av data dir : /opt/conda/etc/conda
  conda av metadata url : None
           channel URLs : https://conda.anaconda.org/conda-forge/linux-64
                          https://conda.anaconda.org/conda-forge/noarch
          package cache : /opt/conda/pkgs
                          /home/jovyan/.conda/pkgs
       envs directories : /opt/conda/envs
                          /home/jovyan/.conda/envs
               platform : linux-64
             user-agent : conda/22.11.1 requests/2

In [2]:
### First we need to get some lidar cloud maps

In [5]:
import os
os.chdir('/home/KID/KID_scripts/cloud_mapping/')


In [8]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import pandas as pd
import geopandas as gpd


api = SentinelAPI('Naderschah', 'VuqZzLLcXEv3KS6', 'https://apihub.copernicus.eu/apihub')

In [9]:
# Here we can create our footprint, to creae a file go here: https://geojsoncreator.com/
footprint = geojson_to_wkt(read_geojson('map.geojson'))
cloudcover_max = 15
start_date = date(2020, 1, 1)

# Now we can query (MSI 2A data product)
products = api.query(footprint,
                     date = ('20200101', date(2020, 12, 31)),
                     platformname = 'Sentinel-2',
                     cloudcoverpercentage = (0, cloudcover_max))

Querying products: 100%|██████████| 1419/1419 [00:28<00:00, 46.79product/s]


In [10]:
# Get meta data, as this contains geograpy and cloud percentages its all we actually need
meta = api.to_geodataframe(products)
# Only select level 2A products
meta = meta[meta.title.apply(lambda x:x.startswith('S2B_MSIL2A'))]

In [11]:
# Drop stuff we definetly do not need 
meta.drop(['link','link_alternative','link_icon', 'ondemand', 'vegetationpercentage', 'notvegetatedpercentage','uuid'],inplace=True,axis=1)

In [12]:
import datetime as dt 
# Format date column
meta['Date'] = meta.summary.apply(lambda x: dt.datetime.strptime(x.split(",")[0].strip("Date: ").split('.')[0].strip(' '), "%Y-%m-%dT%H:%M:%S"))


Looking at the TileIds and geometry fields in the meta table, and the tile id kml fiel : S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml

It is clear the the polygons in the geometry section are for some reason lon/lat instead of lat/lon so they need to be inverted

This can be done with the .reverse() attribute on any shapeley loaded geometry object

In [14]:
# Plot on map to see where these data products live 

from wktplot.plots.osm import OpenStreetMapsPlot
from shapely.ops import transform
import shapely

plot = OpenStreetMapsPlot("Open Street Map Plot", save_dir="cloud_free_plot")

# First we need to exchange the ordering of the polyogn coordinates
def flip(x, y):
    """Flips the x and y coordinate values"""
    return y, x

# Use colorcoding to gve cloudcover probab percentage
for idx, row in meta.iterrows():
    # Ignore nan values for now
    red = row.mediumprobacloudspercentage*cloudcover_max/255 //1
    blue = row.highprobacloudspercentage*cloudcover_max/255 //1
    #plot.add_shape(transform(flip, row.geometry), fill_color=(red, 0, 0, 0.5))
    plot.add_shape(transform(flip, row.geometry), fill_alpha=0.25, fill_color="firebrick")
perim = transform(flip,shapely.wkt.loads(footprint))

plot.add_shape(transform(flip,perim), fill_alpha=0.25, fill_color="blue") 

# The plot can be opened in a browser or something that can read html
plot.save()

In [11]:
# Too determine when all area is covered we will create a cube array that will be populated with nan values around the footprint
# after all elements beloning to the footprint will be set to 0 and then interatively overwritten with the earliest clear observation
# Using the first index and the last index we cna then find the minimum itme required to cover the entire surface area

In [15]:
# Get outer perimeter:
import shapely
perim = transform(flip,shapely.wkt.loads(footprint))

In [16]:
minx, miny, maxx, maxy = perim.bounds
scaler = 50
minx= int((minx)//1)  
miny= int((miny)//1)  
maxx= int((maxx)//1+1)
maxy= int((maxy)//1+1)

In [17]:
import numpy as np
date_array = np.empty(shape=((maxx-minx)*scaler, scaler*(maxy-miny)),dtype='datetime64[ns]')
date_array[:] = np.datetime64("NaT")

In [25]:
from suntime import Sun
# Really inefficient need to find a better way of doing this
for x in range(len(date_array)):
    for y in range(len(date_array[x])):
        # reestablish coordinate
        lat = y/scaler + miny
        lon = x/scaler + minx
        # Check if they intersect with any polygon in the geopandas array
        bool_series = meta.geometry.contains(shapely.Point(lat,lon))
        # Grab dates
        # TODO: the below evaluates to nan for all where cloudcoverpercentage is not nan need to find out details before moving on
        if len(meta[bool_series].Date.dropna())>0 : 
            # Now we check if the date is a night time or daytime obsrevation
            sun=Sun(lat,lon)
            # Turn the Date (type datetime) column to type date 
            dat = meta[bool_series].Date.dropna()
            # Create second bool refering to the ordering of meta[bool_series].Date.dropna()
            second_bool = dat.apply(lambda x: sun.get_sunset_time(x.date()).timestamp() < x.timestamp() < sun.get_sunrise_time(x.date()).timestamp())
            # Check if something is left an move on
            if len(meta[bool_series].Date.dropna()[second_bool])>0:
                date_array[x,y] = meta[bool_series].Date.dropna()[second_bool].dropna().min().to_numpy().astype('datetime64[ns]')
        

In [38]:
date_range = date_array.flatten()
date_range = date_range[~np.isnan(date_range)]
print("Starting obs on the: {}".format(start_date))
print("Earliest Possible: {}".format(np.datetime_as_string(date_range.min(), unit='D')))
print("Earliest time of full observation of footprint: {}".format(np.datetime_as_string(date_range.max(), unit='D')))

Starting obs on the: 2020-01-01
Earliest Possible: 2020-01-06
Earliest time of full observation of footprint: 2020-05-28


'2020-05-28'